In [1]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

In [2]:
CONSUMER_KEY="*addyourkeyhere"
CONSUMER_SECRET= "addyourkeyhere"
ACCESS_TOKEN_KEY = "addyourkeyhere"
ACCESS_TOKEN_SECRET= "addyourkeyhere"

In [3]:
# Create the authentication object
authenticate = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

# Set the access token und access token secret
authenticate.set_access_token(ACCESS_TOKEN_KEY, ACCESS_TOKEN_SECRET)

# Create the API object while passing in the auth information
api = tweepy.API(authenticate, wait_on_rate_limit=True)

In [ ]:
# Extract 100 tweets from the twitter user
twitter_name = input("From which twitter account you want to extract tweets? Pls type in the correct twitter name:  ")
number_of_tweets= input("How many of the recent tweets of the twitter user do you want to extract?")


posts = api.user_timeline(screen_name=twitter_name, count=int(number_of_tweets), lang="en", tweet_mode="extended")

# Print the last 5 tweets from the account
print("Show the 5 recent tweets: \n")
i=1
for tweet in posts[0:5]:
    print(str(i)+ ") "+tweet.full_text + "\n")
    i= i+1
    


In [ ]:
# Create a dataframe with a column called Tweets
df = pd.DataFrame([tweet.full_text for tweet in posts], columns=["Tweets"])
df1 = pd.DataFrame([twitter_name for tweet in posts], columns=["Twittered_By"])
df2 = pd.DataFrame([tweet.created_at for tweet in posts], columns=["Tweet_Date"])

# Show the first 5 rows of data
df2.head()

In [ ]:
# connect all 3 dataframes by column
tbd = pd.concat([df, df1,df2], axis=1)
tbd.head(5)

# tbd for tweet,by,date

In [ ]:
# Clean the text

# Create a function to clean the tweets

def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9]+', '', text) 
    text=re.sub(r'#', '', text)                     
    text=re.sub(r'RT[\s]+', '', text)
    text= re.sub(r'https?:/\/\\S+', '', text)
    text=re.sub(r'https:.*?', '', text)
    text=re.sub(r'^:', '', text)
    
    return text

# Cleaning the text                        
tbd["Tweets"] = tbd["Tweets"].apply(cleanTxt)

# Show cleaned text
tbd

In [ ]:
# Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Create two new columns
tbd["Subjectivity"] = tbd["Tweets"].apply(getSubjectivity)
tbd["Polarity"] = tbd["Tweets"].apply(getPolarity)

# Show the new dataframe with the new columns
tbd

In [ ]:
 #Plot World Cloud
allWords = " ".join([twts for twts in tbd["Tweets"]])
wordCloud= WordCloud(width=500, height=300, random_state=21, max_font_size=119).generate(allWords)

plt.imshow(wordCloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Create a function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score <0:
        return "Negative"
    elif score == 0:
        return "Neutral"
    else:
        return "Positive"
    
tbd["Analysis"] = tbd["Polarity"].apply(getAnalysis)
    
#Show the dataframe

tbd

In [ ]:
# Print all of the positive tweets
j=1
sortedDF=tbd.sort_values(by=["Polarity"])
for i in range(0, sortedDF.shape[0]):
    if (sortedDF["Analysis"][i]=="Positive"):
        print(str(j) + ") "+sortedDF["Tweets"][i])
        print()
        j=j+1

In [ ]:
# Plot the polarity and subjectivity
plt.figure(figsize=(8,6))
for i in range(0, tbd.shape[0]):
    plt.scatter(tbd["Polarity"][i], tbd["Subjectivity"][i], color="Blue")
    
plt.title("Sentiment Analysis")
plt.xlabel("Polarity")
plt.ylabel("Subjectivity")
plt.show()

In [ ]:
# Get the percentage of positive tweets
ptweets = tbd[tbd.Analysis=="Positive"]
ptweets = ptweets["Tweets"]

round( (ptweets.shape[0] / tbd.shape[0])*100, 1)

In [ ]:
#Get the percentage of negative twwets
ptweets = tbd[tbd.Analysis=="Negative"]
ptweets = ptweets["Tweets"]

round( (ptweets.shape[0] / tbd.shape[0])*100, 1)

In [ ]:
# Show the value counts

tbd["Analysis"].value_counts()

#plot and visualize the counts
plt.title("Sentiment Analysis")
plt.xlabel("Sentiment")
plt.ylabel("Counts")
tbd["Analysis"].value_counts().plot(kind="bar")
plt.show()